In [130]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fergus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fergus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [133]:
# load data from database
database_name = "fergus4.db"
output_model_file = "disaster_model"
engine_name = 'sqlite:///' + database_name
engine = create_engine(engine_name)
df =pd.read_sql("SELECT * FROM messages_table", engine)
X = df['message']
#TEMPORTARY
#y = df.drop(['id','message','original','genre'], axis=1)
y = df[['security', 'military']]

In [106]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### Tokenisation Function

In [4]:
def tokenize(text):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### Machine learning pipeline


In [95]:
DTC = DecisionTreeClassifier(random_state = 11)
model_AdaBoost = AdaBoostClassifier(random_state=96 ,base_estimator = DTC )

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=model_AdaBoost))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

y_test = y_test.astype('float64')

(19635,)
(6545,)
(19635, 2)
(6545, 2)


In [9]:
y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
19552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15616,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5861,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19732,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
pipeline.fit(X_train[:2000], y_train[:2000])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...r='best'),
          learning_rate=1.0, n_estimators=50, random_state=96),
           n_jobs=None))])

In [101]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = y_test.columns
y_pred_df = y_pred_df.astype('float64')
y_pred_df.head(5)

,security,military
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [114]:
def print_score(y_actual, y_pred, measure):
    print("\t\tWeighted Average Scores Over Each Output Class\n")
    print("\t\tPrecision\tRecall\t\tF1_Score")
    for column_name, column in y_actual.iteritems():
        report  = classification_report(y_actual[column_name], y_pred[column_name], output_dict=True )
        prec = report[measure]['precision']
        recall =  report[measure]['recall']
        f1 = report[measure]['f1-score']
        print("%20.2f %15.2f % 15.2f" % (prec, recall, f1) + "\t\t" + column_name )

In [122]:
print_score(y_test, y_pred_df, 'weighted avg')

		Weighted Average Scores Over Each Output Class

		Precision	Recall		F1_Score
                0.97            0.97            0.97		security
                0.95            0.96            0.96		military


In [112]:
report

{'0.0': {'precision': 0.9752079083634081,
  'recall': 0.9813674403915995,
  'f1-score': 0.9782779789076027,
  'support': 6333},
 '1.0': {'precision': 0.313953488372093,
  'recall': 0.25471698113207547,
  'f1-score': 0.28125,
  'support': 212},
 'micro avg': {'precision': 0.9578304048892284,
  'recall': 0.9578304048892284,
  'f1-score': 0.9578304048892284,
  'support': 6545},
 'macro avg': {'precision': 0.6445806983677506,
  'recall': 0.6180422107618375,
  'f1-score': 0.6297639894538014,
  'support': 6545},
 'weighted avg': {'precision': 0.9537891250115123,
  'recall': 0.9578304048892284,
  'f1-score': 0.9557004492623145,
  'support': 6545}}

### 6. Improve your model
Use grid search to find better parameters. 

In [111]:
parameters = {'clf__estimator__learning_rate': [0.05, 0.07, 0.1, 0.15, 0.3, 1], 
              'clf__estimator__n_estimators': randint(50, 300),
              'clf__estimator__base_estimator__max_depth': randint(3, 6),
              'clf__estimator__base_estimator__min_samples_split': randint(2,6),
              'clf__estimator__algorithm': ["SAMME", "SAMME.R"]}

gridsearch_percent_of_dataset = 20

len_full_dataset = len(X_train)
sample_size = int((len_full_dataset/100)*gridsearch_percent_of_dataset)
print(len_full_dataset)
print(sample_size)

#scoring='f1_weighted'
grid_obj = RandomizedSearchCV(pipeline,parameters,n_iter=5, cv=5 )
grid_obj.fit(X_train[:sample_size], y_train[:sample_size])

19635
3927


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...r='best'),
          learning_rate=1.0, n_estimators=50, random_state=96),
           n_jobs=None))]),
          fit_params=None, iid='warn', n_iter=5, n_jobs=None,
          param_distributions={'clf__estimator__learning_rate': [0.05, 0.07, 0.1, 0.15, 0.3, 1], 'clf__estimator__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000A1A5763FD0>, 'clf__estimator__base_estimator__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000000A1A60AB160>, 'clf__estimator__base_estimator__min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen obj

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [126]:
predictions.shape

(6545, 2)

In [128]:
optimised_classifier = grid_obj.best_estimator_
#Refit the classifier
optimised_classifier.fit(X_train[:2000], y_train[:2000])
print("fergus")
# Make predictions and score using the optimised model
predictions = optimised_classifier.predict(X_test)
predictions = pd.DataFrame(predictions)
predictions.columns = y_test.columns
predictions = predictions.astype('float64')


print_score(y_test, predictions, 'weighted avg')

fergus
		Weighted Average Scores Over Each Output Class

		Precision	Recall		F1_Score
                0.99            0.98            0.98		security
                0.95            0.97            0.95		military


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [134]:
pickle.dump(optimised_classifier, open(output_model_file, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.